# Librerías

In [1]:
import os

# Librerias base para el analisis y manejo de datos
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Librerias con los modelos principales
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids 

# Libnreria para medir los resultados de los modelos
from sklearn.metrics import silhouette_score, adjusted_rand_score, calinski_harabasz_score

# Libreria para graficar
import matplotlib.pyplot as plt

# Estandarizar la data
from sklearn.preprocessing import StandardScaler

# Reducir la dimensionalidad para graficas
from sklearn.manifold import MDS

# metodo para recrear un iterador infinito
from itertools import cycle

# Funciones

In [2]:
def convertir_col_numericas(df, lista_columnas):

    for columna in lista_columnas:

        df[columna] = df[columna].str.replace(',00', '', regex=False)
        df[columna] = df[columna].str.replace('.', '', regex=False)
        df[columna] = df[columna].str.replace(',', '.', regex=False).astype(float)

    return df

def porcentaje_participacion(df, lista_col, col_total):

    for col in lista_col:
        
        df[f'Porc_part_{col}'] = df[col] / df[col_total]

    return df


def crear_grafico_dispersion(modelo_fit, df, y):
    colors = plt.cm.tab20.colors + plt.cm.tab20b.colors
    color_cycle = cycle(colors)
    mds = MDS(n_components=2, dissimilarity = 'euclidean', random_state=123)
    df_transformado = mds.fit_transform(df)

    unique_labels = np.unique(modelo_fit.labels_)

    plt.figure(figsize=(10, 7))
    
    for idx, label in enumerate(unique_labels):
        color = next(color_cycle)  # Obtener el siguiente color del ciclo
        plt.scatter(df_transformado[modelo_fit.labels_ == label, 0], 
                    df_transformado[modelo_fit.labels_ == label, 1], 
                    label=f'Cluster {label}',
                    color=color)
    # Agregar las etiquetas de cada punto
    for i in range(df_transformado.shape[0]):
        plt.text(df_transformado[i, 0], 
                 df_transformado[i, 1], 
                 y.iloc[i, 0],  
                 fontsize=9, 
                 ha='right', 
                 color='black')

    plt.title('MDS Plot of Data with Labels')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.legend()
    plt.show()

# Preparación bases de datos

## Base municipios

In [3]:
# Base Municipios
df_base = pd.read_excel("../Data/DIVIPOLA_Municipios.xlsx")

#
display(df_base.shape)
df_base.head(5)

(1122, 5)

,CodigoD,Departamento,CodigoM,Municipio,tipo
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio


In [4]:
# Economia
df_educacion = pd.read_csv("../Data/archivos_csv/Educacion.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_educacion = df_educacion[df_educacion['Año']==2022]

# 
display(df_educacion.shape)
df_educacion.head()

(26269, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
19295,76,Valle del Cauca,76892,Yumbo,Educación,Acceso a la educación,Cobertura bruta en transición,"111,06",NaN,2022,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
19296,76,Valle del Cauca,76895,Zarzal,Educación,Acceso a la educación,Cobertura bruta en transición,"77,40",NaN,2022,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
19297,76,Valle del Cauca,76890,Yotoco,Educación,Acceso a la educación,Cobertura bruta en transición,"91,32",NaN,2022,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
19298,76,Valle del Cauca,76869,Vijes,Educación,Acceso a la educación,Cobertura bruta en transición,"70,47",NaN,2022,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
19299,76,Valle del Cauca,76863,Versalles,Educación,Acceso a la educación,Cobertura bruta en transición,"66,06",NaN,2022,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


In [5]:
df_educacion['Subcategoría'].unique()

array(['Acceso a la educación', 'Permanencia y rezago', 'Calidad',
       'Acceso a la educación desagregado por sexo'], dtype=object)

In [6]:
lista_educacion = ['Cobertura neta en educación - Total',
                   'Cobertura neta en educación media',
                   'Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)']
                   

In [7]:
df_educacion = df_educacion[df_educacion['Indicador'].isin(lista_educacion)]

In [8]:
# dejar datos por municipio 
df_educacion1 = df_educacion.pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [9]:
df_educacion1 = df_educacion1.rename(columns={
    'Cobertura neta en educación - Total': 'Cobertura_edu_total',
    'Cobertura neta en educación media': 'Cobertura_edu_media',
    'Tasa de repitencia del sector oficial en educación básica y media (Desde transición hasta once)': 'Tasa_repitencia'
})

In [10]:
lista_edu1 = ['Cobertura_edu_total', 'Cobertura_edu_media', 'Tasa_repitencia']
df_educacion1 = convertir_col_numericas(df_educacion1, lista_edu1)

In [11]:
df_educacion1.head(5)

Indicador,Código Entidad,Entidad,Cobertura_edu_total,Cobertura_edu_media,Tasa_repitencia
0,1001,Colombia,91.52,49.74,0.00
1,5000,Antioquia,93.32,52.35,6.49
2,5001,Medellín,97.06,56.18,6.41
3,5002,Abejorral,77.84,42.72,7.48
4,5004,Abriaquí,67.79,31.40,7.48


In [12]:
# Economia
df_InvCientifica = pd.read_csv("../Data/archivos_csv/Inversion_investicacion_ciencia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)
df_InvCientifica = df_InvCientifica[df_InvCientifica['Año']==2021]
# 
display(df_InvCientifica.shape)
df_InvCientifica.head()

(33, 13)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
693,99,Vichada,99000,Vichada,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,00",NaN,2021,12,MinCiencias,Porcentaje
694,97,Vaupés,97000,Vaupés,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,00",NaN,2021,12,MinCiencias,Porcentaje
695,95,Guaviare,95000,Guaviare,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,00",NaN,2021,12,MinCiencias,Porcentaje
696,94,Guainía,94000,Guainía,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,00",NaN,2021,12,MinCiencias,Porcentaje
697,91,Amazonas,91000,Amazonas,"Ciencia, Tecnología e Innovación","Ciencia, Tecnología e Innovación",Inversión departamental en Investigación y Des...,"0,00",NaN,2021,12,MinCiencias,Porcentaje


In [13]:
df_InvCientifica['Indicador'].unique()

array(['Inversión departamental en Investigación y Desarrollo (I+D) como porcentaje del Producto Interno Bruto (PIB)'],
      dtype=object)

In [14]:
# dejar datos por municipio / son 23 años
df_InvCientifica1 = df_InvCientifica.pivot(
    index=['Código Departamento', 'Departamento'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [15]:
df_InvCientifica1 = df_InvCientifica1.rename(columns={
    'Inversión departamental en Investigación y Desarrollo (I+D) como porcentaje del Producto Interno Bruto (PIB)': 'Inv_Ciencia'
})

In [16]:
df_InvCientifica1 = convertir_col_numericas(df_InvCientifica1, ['Inv_Ciencia'])

In [17]:
df_InvCientifica1.head(5)

Indicador,Código Departamento,Departamento,Inv_Ciencia
0,1,Colombia,NaN
1,5,Antioquia,0.05
2,8,Atlántico,0.01
3,13,Bolívar,0.01
4,15,Boyacá,0.00


## Base Inclusion financiera

In [18]:
# Base Municipios
df_inclusion = pd.read_csv("../Data/Inclusi_n_Financiera_20250515.csv")

#
display(df_inclusion.shape)
df_inclusion.head(5)

(367775, 99)

,TIPO_ENTIDAD,CODIGO_ENTIDAD,NOMBRE_ENTIDAD,FECHA_CORTE,UNICAP,DESCRIP_UC,RENGLON,DESC_RENGLON,TIPO,(1) NRO_CORRESPONSALES_FÍSICOS_PROPIOS,(2) NRO_CORRESPONSALES_FÍSICOS_TERCERIZADOS,(3) NRO_CORRESPONSALES_FÍSICOS_ACTIVOS,(4) NRO_CORRESPONSALES_FÍSICOS,(5) NRO_DEPÓSITOS_CORRESPONSALES_FÍSICOS,(6) MONTO_DEPÓSITOS_CORRESPONSALES_FÍSICOS,(7) NRO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS,(8) MONTO_GIROS_ENVIADOS_CORRESPONSALES_FÍSICOS,(9) NRO_GIROS_RECIBIDOS_CORRESPONSALES_FÍSICOS,(10) MONTO_GIROS_RECIBIDOS_CORRESPONSALES_FÍSICOS,(11) NRO_PAGOS_CORRESPONSALES_FÍSICOS,(12) MONTO_PAGOS_CORRESPONSALES_FÍSICOS,(13) NRO_RETIROS_CORRESPONSALES_FÍSICOS,(14) MONTO_RETIROS_CORRESPONSALES_FÍSICOS,(15) NRO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS,(16) MONTO_TRANSFERENCIAS_CORRESPONSALES_FÍSICOS,(17) NRO_TRANSACCIONES_TRÁMITES_CORRESPONSALES_FÍSICOS,(18) MONTO_TRANSACCIONES_CORRESPONSALES_FÍSICOS,(19) NRO_CTAS_AHORRO_HASTA_1SMMLV,(20) SALDO_CTAS_AHORRO_HASTA_1SMMLV,(21) NRO_CTAS_AHORRO>1SMMLV_HASTA _3SMMLV,(22) SALDO_CTAS_AHORRO>1SMMLV_HASTA _3SMMLV,(23) NRO_CTAS_AHORR>3SMMLV_HASTA_ 5SMMLV,(24) SALDO_CTAS_AHORR>3SMMLV_HASTA_ 5SMMLV,(25) NRO_CTAS_AHORRO_ACTIVAS,(26) SALDO_CTAS_AHORRO_ACTIVAS,(27) NRO_CTAS_AHORRO_MUJERES,(28) SALDO_CTAS_AHORRO_MUJERES,(29) NRO_CTAS_AHORRO_HOMBRES,(30) SALDO_CTAS_AHORRO_HOMBRES,(31) NRO_CTAS_AHORRO,(32) SALDO_CTAS_AHORRO,(41) NRO_CRÉDITO_CONSUMO_MUJERES,(42) MONTO_CRÉDITO_CONSUMO_MUJERES,(43) NRO_CRÉDITO_CONSUMO_HOMBRES,(44) MONTO_CRÉDITO_CONSUMO_HOMBRES,(45) NRO_CRÉDITO_CONSUMO,(46) MONTO_CRÉDITO_CONSUMO,(47) NRO_CRED_CONS_BAJO_MONTO_MUJERES,(48) MONTO_CRED_CONS_BAJO_MONTO_MUJERES,(49) NRO_CRED_CONS_BAJO_MONTO_HOMBRES,(50) MONTO_CRED_CONS_BAJO_MONTO_HOMBRES,(51) NRO_CRED_CONS_BAJO_MONTO,(52) MONTO_CRED_CONS_BAJO_MONTO,(53) NRO_CRÉDITO_VIVIENDA_MUJERES,(54) MONTO_CRÉDITO_VIVIENDA_MUJERES,(55) NRO_CRÉDITO_VIVIENDA_HOMBRES,(56) MONTO_CRÉDITO_VIVIENDA_HOMBRES,(57) NRO_CRÉDITO_VIVIENDA,(58) MONTO_CRÉDITO_VIVIENDA,(59) NRO_MICROCRÉDITO_HASTA_ 1SMMLV,(60) MONTO_MICROCRÉDITO_HASTA_1SMMLV,(61) NRO_MICROCRÉDITO_>1SMMLV_HASTA_ 2SMMLV,(62) MONTO_MICROCRÉDITO_>1SMMLV HASTA_ 2SMMLV,(63) NRO_MICROCRÉDITO_>2SMMLV_HASTA_3SMMLV,(64) MONTO_MICROCRÉDITO_> 2SMMLV_HASTA_3SMMLV,(65) NRO_MICROCRÉDITO_>3SMMLV_HASTA_ 4SMMLV,(66) MONTO_MICROCRÉDITO_>3SMMLV_HASTA_4SMMLV,(67) NRO_MICROCRÉDITO_>4SMMLV_HASTA_10SMMLV,(68) MONTO_MICROCRÉDITO_> 4SMMLV_HASTA_10SMMLV,(69) NRO_MICROCRÉDITO_>10SMMLV_HASTA_25SMMLV,(70) MONTO_MICROCRÉDITO_>10SMMLV_HASTA_ 25SMMLV,(71) NRO_MICROCRÉDITO_MUJERES,(72) MONTO_MICROCRÉDITO_MUJERES,(73) NRO_MICROCRÉDITO_HOMBRES,(74) MONTO_MICROCRÉDITO_HOMBRES,(75) NRO_MICROCRÉDITO,(76) MONTO_MICROCRÉDITO,(77) NRO_PRODUCTOS_A_NIVEL_NACIONAL,(78) MONTO_SALDO_PRODUCTOS_A_NIVEL_NACIONAL,(79) NRO_CORRESPONSALES_FÍSICOS_PROPIOS_ACTIVOS,(80) NRO_CORRESPONSALES_FÍSICOS_TERCERIZADOS_ACTIVOS,(81) NRO_CORRESPONSALES_PROPIOS_MÓVILES,(82) NRO_CORRESPONSALES_TERCERIZADOS_MÓVILES,(83) NRO_CORRESPONSALES_MÓVILES_ACTIVOS,(84) NRO_CORRESPONSALES_MÓVILES,(85) NRO_DEPÓSITOS_CORRESPONSALES_MÓVILES,(86) MONTO_DEPÓSITOS_CORRESPONSALES_MÓVILES,(87) NRO_GIROS_ENVIADOS_CORRESPONSALES_MÓVILES,(88) MONTO_GIROS_ENVIADOS_CORRESPONSALES_MÓVILES,(89) NRO_GIROS_RECIBIDOS_CORRESPONSALES_MÓVILES,(90) MONTO_GIROS_RECIBIDOS_CORRESPONSALES_MÓVILES,(91) NRO_PAGOS_CORRESPONSALES_MÓVILES,(92) MONTO_PAGOS_CORRESPONSALES_MÓVILES,(93) NRO_RETIROS_CORRESPONSALES_MÓVILES,(94) MONTO_RETIROS_CORRESPONSALES_MÓVILES,(95) NRO_TRANSFERENCIAS_CORRESPONSALES_MÓVILES,(96) MONTO_TRANSFERENCIAS_CORRESPONSALES_MÓVILES,(97) NRO_TRANSACCIONES_TRÁMITES_CORRESPONSALES_MÓVILES,(98) MONTO_TRANSACCIONES_CORRESPONSALES_MÓVILES
0,4,46,Coltefinanciera,31/03/2024,19,QUINDIO,999,TOTAL QUINDÍO,Corresponsales fisicos,1,9,10,10,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0.00,0,0,0,0.00
1,1,59,Banco 

In [19]:
df_inclusion['TIPO'].unique()

array(['Corresponsales fisicos', 'Cuentas de Ahorro',
       'Corresponsales físicos',
       'Transacciones y tramites a traves de corresponsales fisicos',
       'Microcredito por rango', 'Credito de consumo',
       'Transacciones corresponsal movil', 'Microcredito',
       'Productos a nivel nacional', 'Credito de consumo de bajo monto',
       'Credito de vivienda', 'Numero de corresponsales moviles'],
      dtype=object)

In [20]:
# Se eliminan registros no objeto de estudio
col_eliminar = ['Corresponsales fisicos', 'Corresponsales físicos', 'Productos a nivel nacional', 'Microcredito por rango']

In [21]:
df_inclusion1 = df_inclusion[~df_inclusion['TIPO'].isin(col_eliminar)]
df_inclusion1 = df_inclusion1.drop(columns=[col for col in df_inclusion1.columns if 'HOMBRES' in col or 'MUJERES' in col or 'DEPÓSITOS' in col
                                           or 'GIROS' in col or 'PAGOS' in col or 'RETIROS' in col or 'TRANSFERENCIAS' in col or 'SMMLV' in col
                                           or 'ACTIVAS' in col or 'ACTIVOS' in col or 'PROPIOS' in col or 'TERCERIZADOS' in col])

In [22]:
df_inclusion1['Total_Numero'] = df_inclusion1[[col for col in df_inclusion1.columns if 'NRO' in col]].sum(axis=1)
df_inclusion1['Total_Monto'] = df_inclusion1[[col for col in df_inclusion1.columns if 'MONTO' in col]].sum(axis=1)
df_inclusion1['Total_Saldo'] = df_inclusion1[[col for col in df_inclusion1.columns if 'SALDO' in col]].sum(axis=1)

In [23]:
df_inclusion1.head(10)

,TIPO_ENTIDAD,CODIGO_ENTIDAD,NOMBRE_ENTIDAD,FECHA_CORTE,UNICAP,DESCRIP_UC,RENGLON,DESC_RENGLON,TIPO,(4) NRO_CORRESPONSALES_FÍSICOS,(17) NRO_TRANSACCIONES_TRÁMITES_CORRESPONSALES_FÍSICOS,(18) MONTO_TRANSACCIONES_CORRESPONSALES_FÍSICOS,(31) NRO_CTAS_AHORRO,(32) SALDO_CTAS_AHORRO,(45) NRO_CRÉDITO_CONSUMO,(46) MONTO_CRÉDITO_CONSUMO,(51) NRO_CRED_CONS_BAJO_MONTO,(52) MONTO_CRED_CONS_BAJO_MONTO,(57) NRO_CRÉDITO_VIVIENDA,(58) MONTO_CRÉDITO_VIVIENDA,(75) NRO_MICROCRÉDITO,(76) MONTO_MICROCRÉDITO,(77) NRO_PRODUCTOS_A_NIVEL_NACIONAL,(78) MONTO_SALDO_PRODUCTOS_A_NIVEL_NACIONAL,(84) NRO_CORRESPONSALES_MÓVILES,(97) NRO_TRANSACCIONES_TRÁMITES_CORRESPONSALES_MÓVILES,(98) MONTO_TRANSACCIONES_CORRESPONSALES_MÓVILES,Total_Numero,Total_Monto,Total_Saldo
2,1,30,Banco Caja Social S.A.,31/03/2024,11,CUNDINAMARCA,126,CAJICA,Cuentas de Ahorro,0,0,0.00,43198,41530976062.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,43198,0.00,41530976062.00
4,1,43,Banagrario,31/03/2024,2,ATLANTICO,560,PONEDERA,Transacciones y tramites a traves de correspon...,0,3142,1971644602.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,3142,1971644602.00,0.00
9,1,39,Banco Davivienda,31/03/2024,8,CAUCA,364,JAMBALO,Transacciones y tramites a traves de correspon...,0,57,13206705.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,57,13206705.00,0.00
12,1,57,Banco Pichincha S.A.,31/03/2024,6,CALDAS,999,TOTAL CALDAS,Credito de consumo,0,0,0.00,0,0.00,47,2017125453.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,47,2017125453.00,0.00
13,1,43,Banagrario,31/03/2024,11,CUNDINAMARCA,372,JUNIN,Cuentas de Ahorro,0,0,0.00,2976,6991234620.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,2976,0.00,6991234620.00
17,32,5,Confiar,31/03/2024,5,BOYACA,798,TENZA,Transacciones corresponsal movil,0,0,0.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,0,0.00,0.00
18,1,39,Banco Davivienda,31/03/2024,17,NARIÑO,215,CORDOBA,Transacciones y tramites a traves de correspon...,0,1052,172835717.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,1052,172835717.00,0.00
19,4,123,Bancar Tecnología C.F.,31/03/2024,24,VALLE DEL CAUCA,606,RESTREPO,Transacciones y tramites a traves de correspon...,0,2,350000.00,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,2,350000.00,0.00
24,4,117,Credifamilia,31/03/2024,3,BOGOTA,1,SANTAFE DE BOGOTA D.,Transacciones y tramites a traves de correspon...,0,15474,10831417760.32,0,0.00,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,15474,10831417760.32,0.00
25,1,42,Scotiabank Colpatria S.A.,31/03/2024,8,CAUCA,1,POPAYAN,Cuentas de Ahorro,0,0,0.00,15201,99831825360.22,0,0.00,0,0,0,0.00,0,0.00,0,0.00,0,0,0.00,15201,0.00,99831825360.22


In [24]:
# Paso 1: Crear dataframe base con valores únicos de DESCRIP_UC y DESC_RENGLON
df_inclusion2 = df_inclusion1[['DESCRIP_UC', 'DESC_RENGLON']].drop_duplicates().reset_index(drop=True)

# Paso 2: Iterar sobre cada valor único de TIPO
tipos_unicos = df_inclusion1['TIPO'].unique()

# Paso 3: Por cada TIPO, agrupar y renombrar columnas, luego hacer merge con df_base
for tipo in tipos_unicos:
    df_tmp = (
        df_inclusion1[df_inclusion1['TIPO'] == tipo]
        .groupby(['DESCRIP_UC', 'DESC_RENGLON'], as_index=False)[['Total_Numero', 'Total_Monto', 'Total_Saldo']]
        .sum()
    )
    
    # Paso 4: Renombrar columnas incluyendo el tipo
    df_tmp = df_tmp.rename(columns={
        'Total_Numero': f'Total_Numero_{tipo}',
        'Total_Monto': f'Total_Monto_{tipo}',
        'Total_Saldo': f'Total_Saldo_{tipo}'
    })
    
    # Paso 5: Merge al dataframe base
    df_inclusion2 = df_inclusion2.merge(df_tmp, on=['DESCRIP_UC', 'DESC_RENGLON'], how='left')

# Resultado final: df_base contiene los totales por cada tipo en columnas separadas


In [25]:
df_inclusion2.head(5)

,DESCRIP_UC,DESC_RENGLON,Total_Numero_Cuentas de Ahorro,Total_Monto_Cuentas de Ahorro,Total_Saldo_Cuentas de Ahorro,Total_Numero_Transacciones y tramites a traves de corresponsales fisicos,Total_Monto_Transacciones y tramites a traves de corresponsales fisicos,Total_Saldo_Transacciones y tramites a traves de corresponsales fisicos,Total_Numero_Credito de consumo,Total_Monto_Credito de consumo,Total_Saldo_Credito de consumo,Total_Numero_Transacciones corresponsal movil,Total_Monto_Transacciones corresponsal movil,Total_Saldo_Transacciones corresponsal movil,Total_Numero_Microcredito,Total_Monto_Microcredito,Total_Saldo_Microcredito,Total_Numero_Credito de consumo de bajo monto,Total_Monto_Credito de consumo de bajo monto,Total_Saldo_Credito de consumo de bajo monto,Total_Numero_Credito de vivienda,Total_Monto_Credito de vivienda,Total_Saldo_Credito de vivienda,Total_Numero_Numero de corresponsales moviles,Total_Monto_Numero de corresponsales moviles,Total_Saldo_Numero de corresponsales moviles
0,CUNDINAMARCA,CAJICA,658406.00,0.00,1411408475005.87,1458152,512970951855.55,0.00,227990.00,190360743120.59,0.00,150.00,31844847.00,0.00,967.00,14622740512.19,0.00,0.00,0.00,0.00,436.00,57566457312.58,0.00,15.00,0.00,0.00
1,ATLANTICO,PONEDERA,489.00,0.00,58924254.26,120755,43006240891.49,0.00,703.00,161350593.30,0.00,56.00,8821957.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,0.00
2,CAUCA,JAMBALO,15396.00,0.00,44340305658.85,38996,17611943093.00,0.00,98.00,592142964.28,0.00,0.00,0.00,0.00,584.00,4724967826.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,CALDAS,TOTAL CALDAS,6342548.00,0.00,10809986121402.48,16734970,6770021531407.32,0.00,1438335.00,1788709787080.19,0.00,118767.00,31196446163.00,0.00,27664.00,222631986137.58,0.00,0.00,0.00,0.00,2759.00,345598133641.79,0.00,167.00,0.00,0.00
4,CUNDINAMARCA,JUNIN,12286.00,0.00,26766501997.40,68189,16444505526.00,0.00,228.00,651392100.65,0.00,NaN,NaN,NaN,360.00,3959467051.31,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Se eliminan las siguientes columnas ya que dada la transformacion siempre van a ser NA o 0
col_no_validas = ['Total_Monto_Cuentas de Ahorro', 'Total_Saldo_Transacciones y tramites a traves de corresponsales fisicos',
                  'Total_Saldo_Credito de consumo', 'Total_Saldo_Transacciones corresponsal movil', 'Total_Saldo_Microcredito',
                  'Total_Saldo_Credito de consumo de bajo monto', 'Total_Saldo_Credito de vivienda', 'Total_Monto_Numero de corresponsales moviles',
                  'Total_Saldo_Numero de corresponsales moviles']

In [27]:
df_inclusion3 = df_inclusion2.drop(columns=col_no_validas)

In [28]:
df_inclusion3.head(5)

,DESCRIP_UC,DESC_RENGLON,Total_Numero_Cuentas de Ahorro,Total_Saldo_Cuentas de Ahorro,Total_Numero_Transacciones y tramites a traves de corresponsales fisicos,Total_Monto_Transacciones y tramites a traves de corresponsales fisicos,Total_Numero_Credito de consumo,Total_Monto_Credito de consumo,Total_Numero_Transacciones corresponsal movil,Total_Monto_Transacciones corresponsal movil,Total_Numero_Microcredito,Total_Monto_Microcredito,Total_Numero_Credito de consumo de bajo monto,Total_Monto_Credito de consumo de bajo monto,Total_Numero_Credito de vivienda,Total_Monto_Credito de vivienda,Total_Numero_Numero de corresponsales moviles
0,CUNDINAMARCA,CAJICA,658406.00,1411408475005.87,1458152,512970951855.55,227990.00,190360743120.59,150.00,31844847.00,967.00,14622740512.19,0.00,0.00,436.00,57566457312.58,15.00
1,ATLANTICO,PONEDERA,489.00,58924254.26,120755,43006240891.49,703.00,161350593.30,56.00,8821957.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00
2,CAUCA,JAMBALO,15396.00,44340305658.85,38996,17611943093.00,98.00,592142964.28,0.00,0.00,584.00,4724967826.65,0.00,0.00,0.00,0.00,0.00
3,CALDAS,TOTAL CALDAS,6342548.00,10809986121402.48,16734970,6770021531407.32,1438335.00,1788709787080.19,118767.00,31196446163.00,27664.00,222631986137.58,0.00,0.00,2759.00,345598133641.79,167.00
4,CUNDINAMARCA,JUNIN,12286.00,26766501997.40,68189,16444505526.00,228.00,651392100.65,NaN,NaN,360.00,3959467051.31,0.00,0.00,NaN,NaN,NaN


## Saldos por municipio

In [29]:
df_saldos = pd.read_csv("../Data/Saldo_de_las_captaciones_y_colocaciones_por_municipios_20250515.csv")

#
display(df_saldos.shape)
df_saldos.head(5)

(25146, 80)

,Tipo de entidad,Código de entidad,Código del departamento,Código del municipio,Nombre del municipio,Fecha de Corte,Depósitos en cuenta corriente bancaria,Depósitos simples,Certificados de depósito a término,Depósitos de ahorro,Cuenta de ahorros de valor real,Cuentas de ahorro especial,Certificado de ahorro valor real,Documentos por pagar,Cuenta centralizada,Títulos de inversión en circulación,Cartera de créditos,Créditos de vivienda,Categoría A riesgo normal,Categoría B riesgo aceptable,Categoría C riesgo apreciable,Categoría D riesgo significativo,Categoría E riesgo de Incobrabilidad,"Categoría A riesgo normal, garantía idónea, créditos y leasing de consumo","Categoría B riesgo aceptable, garantía idónea, créditos y leasing de consumo","Categoría C riesgo apreciable, garantía idónea, créditos y leasing de consumo","Categoría D riesgo significativo, garantía idónea, créditos y leasing de consumo","Categoría E riesgo de incobrabilidad, garantía idónea, créditos y leasing de consumo","Categoría A riesgo normal, otras garantías, créditos y leasing de consumo","Categoría B riesgo aceptable, otras garantías, créditos y leasing de consumo","Categoría C riesgo apreciable, otras garantías, créditos y leasing de consumo","Categoría D riesgo significativo, otras garantías, créditos y leasing de consumo","Categoría E riesgo de incobrabilidad, otras garantías, créditos y leasing de consumo","Microcréditos, garantía idónea",Categoría A riesgo normal C29,Categoría B riesgo aceptable C30,Categoría C riesgo apreciable C31,Categoría D riesgo significativo C32,Categoría E riesgo de Incobrabilidad C33,Categoría A riesgo normal bienes dados en leasing C34,Categoría B riesgo aceptable bienes dados en leasing,Categoría C riesgo apreciable bienes dados en leasing,Categoría D riesgo significativo bienes dados en leasing,Categoría E riesgo de Incobrabilidad bienes dados en leasing,"Microcréditos, otras garantías",Categoría A riesgo normal C40,Categoría B riesgo aceptable C41,Categoría C riesgo apreciable C42,Categoría D riesgo significativo C43,Categoría E riesgo de Incobrabilidad C44,Categoría A riesgo normal bienes dados en leasing C45,Categoría B riesgo aceptable bienes dados en leasin C46,Categoría C riesgo apreciable bienes dados en leasing C47,Categoría D riesgo significativo bienes dados en leasing C48,Categoría E riesgo de Incobrabilidad bienes dados en leasing C49,"Categoría A riesgo normal, garantía idónea, créditos y leasing comerciales","Categoría B riesgo aceptable, garantía idónea, créditos y leasing comerciales","Categoría C riesgo apreciable, garantía idónea, créditos y leasing comerciales","Categoría D riesgo significativo, garantía idónea, créditos y leasing comerciales","Categoría E riesgo de incobrabilidad, garantía idónea, créditos y leasing comerciales","Categoría A riesgo normal, otras garantías, créditos y leasing comerciales","Categoría B riesgo aceptable, otras garantías, créditos y leasing comerciales","Categoría C riesgo apreciable, otras garantías, créditos y leasing comerciales","Categoría D riesgo significativo, otras garantías, créditos y leasing comerciales","Categoría E riesgo de incobrabilidad, otras garantías, créditos y leasing comerciales",Provisión créditos de vivienda,Provisión créditos y operaciones de leasing de consumo,Provisión microcréditos,Provisión créditos y operaciones de leasing comerciales,Provisión general,Número de CDT,Número de cuentas centralizadas,Número de oficinas,Número de empleados contratados,Número de empleados subcontratados,Componente contracíclico provisión individual,Derechos de transferencia de cartera de créditos por operaciones de apoyos transitorios de liquidez,Préstamos a empleados,Deterioro (provisión) préstamos a empleados,Crédito de consumo de bajo monto
0,1,54,2,999,TOTAL ATLANTICO,31/12/2024,13720536515.29,0.00,95730055626.39,50706442658.93,0,1387137819.89,0.00,0.00,0,0.00,369418264918.10,94732135062.00,87805425538.00,1621802675.00,1215884648.00,1736011736.00,2353010465.00,13

In [32]:
# Se eliminan variables no objeto de estudio
col_eliminar2 = ['Depósitos simples', 'Cuenta de ahorros de valor real', 'Certificado de ahorro valor real',
                 'Documentos por pagar', 'Cuenta centralizada', 'Títulos de inversión en circulación', 'Número de cuentas centralizadas',
                 'Número de oficinas', 'Número de empleados contratados', 'Número de empleados subcontratados', 'Componente contracíclico provisión individual',
                 'Derechos de transferencia de cartera de créditos por operaciones de apoyos transitorios de liquidez', 'Préstamos a empleados',
                 'Deterioro (provisión) préstamos a empleados']

In [33]:
df_saldos1 = df_saldos.drop(columns=col_eliminar2)

In [34]:
df_saldos1 = df_saldos1.drop(columns=[col for col in df_saldos1.columns if 'Categoría' in col])

In [35]:
df_saldos1.head(5)

,Tipo de entidad,Código de entidad,Código del departamento,Código del municipio,Nombre del municipio,Fecha de Corte,Depósitos en cuenta corriente bancaria,Certificados de depósito a término,Depósitos de ahorro,Cuentas de ahorro especial,Cartera de créditos,Créditos de vivienda,"Microcréditos, garantía idónea","Microcréditos, otras garantías",Provisión créditos de vivienda,Provisión créditos y operaciones de leasing de consumo,Provisión microcréditos,Provisión créditos y operaciones de leasing comerciales,Provisión general,Número de CDT,Crédito de consumo de bajo monto
0,1,54,2,999,TOTAL ATLANTICO,31/12/2024,13720536515.29,95730055626.39,50706442658.93,1387137819.89,369418264918.10,94732135062.00,0.00,0,2442303675.19,14876291169.74,0.00,6978484729.40,947321350.62,1591,0.00
1,2,11,2,999,TOTAL ATLANTICO,31/12/2024,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,1,0.00
2,1,65,33,999,TOTAL VICHADA,31/12/2024,0.00,0.00,20634423.40,0.00,105004921.06,0.00,0.00,0,0.00,6856226.85,0.00,0.00,0.00,0,0.00
3,4,46,2,999,TOTAL ATLANTICO,31/12/2024,0.00,50466877248.13,1917691630.96,0.00,8884066294.77,0.00,0.00,0,0.00,732091479.11,0.00,0.00,0.00,716,0.00
4,1,56,9,1,VALLEDUPAR,31/12/2024,5213997.00,10033136304.00,10324833854.00,0.00,26485769389.00,0.00,0.00,0,0.00,1570138837.00,0.00,0.00,0.00,128,0.00


## Mercado laboral

In [ ]:
# Economia
df_mercadoLaboral = pd.read_csv("../Data/archivos_csv/Mercado_laboral.csv", sep=";", encoding="ISO-8859-1", low_memory=False)
df_mercadoLaboral = df_mercadoLaboral[df_mercadoLaboral['Año'] == 2016]
# 
display(df_mercadoLaboral.shape)
df_mercadoLaboral.head()

In [ ]:
df_mercadoLaboral['Subcategoría'].unique()

In [ ]:
lista_laboral = ['Porcentaje de personas ocupadas formalmente con respecto a la población total']

In [ ]:
df_mercadoLaboral1 = df_mercadoLaboral[df_mercadoLaboral['Indicador'].isin(lista_laboral)]

In [ ]:
# dejar datos por municipio / son 23 años
df_mercadoLaboral1 = df_mercadoLaboral1.pivot(
    index=['Código Entidad', 'Entidad'],
    columns='Indicador',
    values='Dato Numérico'
).reset_index()

In [ ]:
df_mercadoLaboral1 = convertir_col_numericas(df_mercadoLaboral1, lista_laboral)

In [ ]:
df_mercadoLaboral1 = df_mercadoLaboral1.rename(columns={
    'Porcentaje de personas ocupadas formalmente con respecto a la población total': 'Porc_trabajos_formales'
})

In [ ]:
df_mercadoLaboral1['Porc_trabajos_formales'] = df_mercadoLaboral1['Porc_trabajos_formales'] / 100

In [ ]:
df_mercadoLaboral1.head(5)

## Transaccionalidad

In [ ]:
df_transacciones = pd.read_csv("../Data/archivos_csv/transaccionalidad.csv", sep=",", encoding="utf-8")

In [ ]:
display(df_transacciones.shape)
df_transacciones.head(3)

In [ ]:
# Seleccionamos los codigos de operacion digitales
lista_operaciones_digitales = [1, 6, 7, 8, 13, 18]
df_transacciones = df_transacciones[df_transacciones['CODIGO_CANAL_DISTRIBUCION'].isin(lista_operaciones_digitales)]

In [ ]:
df_transacciones1 = df_transacciones.groupby('CODIGO_MUNICIPIO', as_index=False)[['CANTIDAD_OPERACIONES', 'MONTO_OPERACIONES']].sum()


In [ ]:
df_transacciones1.head(5)

## Indice adopcion digital

### 2016

In [ ]:
df_CNC_2016 = pd.read_csv("../Data/archivos_csv/d2016---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2016.head(5)

In [ ]:
col_2016 = ['PB_DANE', 'INDICADOR']
df_CNC_2016 = df_CNC_2016[col_2016]

In [ ]:
df_CNC_2016['INDICADOR'] = df_CNC_2016['INDICADOR'].str.replace(',', '.', regex=False).astype(float)

In [ ]:
df_CNC_2016.head(3)

In [ ]:
df_CNC_2016_1 = df_CNC_2016.groupby('PB_DANE', as_index=False)[['INDICADOR']].mean()
df_CNC_2016_1['Año_datos'] = 2016

In [ ]:
df_CNC_2016_1.head(3)

### 2018

In [ ]:
df_CNC_2018 = pd.read_csv("../Data/archivos_csv/d2018---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2018.head(5)

In [ ]:
col_2018 = ['PB', 'INDICADOR']
df_CNC_2018 = df_CNC_2018[col_2018]

In [ ]:
df_CNC_2018_1 = df_CNC_2018.groupby('PB', as_index=False)[['INDICADOR']].mean()
df_CNC_2018_1['Año_datos'] = 2018

In [ ]:
df_CNC_2018_1 = df_CNC_2018_1.rename(columns={
    'PB': 'PB_DANE'
})

In [ ]:
df_CNC_2016_1.head(3)

### 2023

In [ ]:
df_CNC_2023 = pd.read_csv("../Data/archivos_csv/d2023---copia.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

df_CNC_2023.head(5)

In [ ]:
col_2023 = ['DANE', 'indicador']
df_CNC_2023 = df_CNC_2023[col_2023]

In [ ]:
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.strip()
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.replace('-', '0', regex=False)
df_CNC_2023['indicador'] = df_CNC_2023['indicador'].str.replace(',', '.', regex=False).astype(float)

In [ ]:
df_CNC_2023_1 = df_CNC_2023.groupby('DANE', as_index=False)[['indicador']].mean()
df_CNC_2023_1['Año_datos'] = 2023

In [ ]:
df_CNC_2023_1 = df_CNC_2023_1.rename(columns={
    'DANE': 'PB_DANE',
    'indicador': 'INDICADOR'
})

In [ ]:
df_CNC_2023_1.head(3)

### Validacion columnas

In [ ]:
print(df_CNC_2016_1.columns, df_CNC_2016_1.shape[0])
print(df_CNC_2018_1.columns, df_CNC_2018_1.shape[0])
print(df_CNC_2023_1.columns, df_CNC_2023_1.shape[0])

### Union todos los datasets

In [ ]:
cnc_concat = pd.concat([df_CNC_2016_1, df_CNC_2018_1, df_CNC_2023_1], ignore_index=True)

In [ ]:
cnc_concat.shape[0]

### Seleccion valor mas actualizado por municipio

In [ ]:
# Ordena por cod_municipio y año (de más reciente a más antiguo)
cnc_concat1 = cnc_concat.sort_values(by=['PB_DANE', 'Año_datos'], ascending=[True, False])

# Elimina duplicados, conservando solo el más reciente por cod_municipio
cnc_concat1 = cnc_concat1.drop_duplicates(subset='PB_DANE', keep='first')

In [ ]:
display(cnc_concat1.shape)
cnc_concat1.head(3)

In [ ]:
cnc_concat_def = cnc_concat1[['PB_DANE', 'INDICADOR']]

# Armado dataframe Final

## Agregar info depto

In [ ]:
df1 = pd.merge(cnc_concat_def, df_base[['CodigoD','CodigoM', 'Municipio']], left_on='PB_DANE', right_on='CodigoM', how='left')

In [ ]:
df1 = df1.drop(['CodigoM'], axis=1)

In [ ]:
display(df1.shape)
df1.head(3)

## Info Finanzas

In [ ]:
df2 = pd.merge(df1, df_finanzas_final[['Código Entidad','Porc_part_Educación','Porc_part_Propósito general']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df2 = df2.drop(['Código Entidad'], axis=1)

In [ ]:
display(df2.shape)
df2.head(3)

## Info economía

In [ ]:
df3 = pd.merge(df2, df_economia_final[['Código Departamento','PIB_per_capita','PIB_act_financieras','PIB_act_prof','PIB_info_comunicaciones']], left_on='CodigoD', right_on='Código Departamento', how='left')

In [ ]:
df3 = df3.drop(['Código Departamento'], axis=1)
df3 = df3.fillna(0)

In [ ]:
display(df3.shape)
df3.head(3)

## Info educación

In [ ]:
df4 = pd.merge(df3, df_educacion1[['Código Entidad','Cobertura_edu_total','Cobertura_edu_media','Tasa_repitencia']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df4 = df4.drop(['Código Entidad'], axis=1)
#df3 = df3.fillna(0)

In [ ]:
display(df4.shape)
df4.head(3)

## Info Inv ciencia

In [ ]:
df5 = pd.merge(df4, df_InvCientifica1[['Código Departamento','Inv_Ciencia']], left_on='CodigoD', right_on='Código Departamento', how='left')

In [ ]:
df5 = df5.drop(['Código Departamento'], axis=1)
df5 = df5.fillna(0)

In [ ]:
display(df5.shape)
df5.head(3)

## Info Mercado Laboral

In [ ]:
df6 = pd.merge(df5, df_mercadoLaboral1[['Código Entidad','Porc_trabajos_formales']], left_on='PB_DANE', right_on='Código Entidad', how='left')

In [ ]:
df6 = df6.drop(['Código Entidad'], axis=1)
#df6 = df6.fillna(0)

In [ ]:
display(df6.shape)
df6.head(3)

## Info Transaccionalidad

In [ ]:
df7 = pd.merge(df6, df_transacciones1, left_on='PB_DANE', right_on='CODIGO_MUNICIPIO', how='left')

In [ ]:
df7 = df7.drop(['CODIGO_MUNICIPIO'], axis=1)

In [ ]:
display(df7.shape)
df7.head(3)

# Construccion Modelo No Supervisado

## Escalado de los datos

In [ ]:
# Seleccion variables
X = df7.drop(['PB_DANE', 'CodigoD', 'Municipio'], axis=1)
y = df7['PB_DANE']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df_X = pd.DataFrame(X_scaled, columns=X.columns)
df_y = pd.DataFrame(y, columns=["PB_DANE"])

In [ ]:
df = df_y.join(df_X)
df.head()

## Desarrollo modelo

### K-Means

In [ ]:
kmeans_1 = KMeans(n_clusters = 5, random_state = 42, n_init=10).fit(df_X)

In [ ]:
# Obtener las etiquetas de cluster generadas por KMeans
cluster_labels_1= kmeans_1.labels_

In [ ]:
crear_grafico_dispersion(kmeans_1, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, kmeans_1.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, kmeans_1.labels_))
print("Metrica de Inercia de k-means: ",kmeans_1.inertia_)

### K-Medoides

In [ ]:
Kmedoides_1 = KMedoids(n_clusters = 5, random_state = 42).fit(df_X)

In [ ]:
cluster_labels_2= Kmedoides_1.labels_

In [ ]:
crear_grafico_dispersion(Kmedoides_1, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, Kmedoides_1.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, Kmedoides_1.labels_))
print("Metrica de Inercia de k-means: ",Kmedoides_1.inertia_)

## Calibracion parámetros

### K-Means

In [ ]:
# Numero de cluster a validar
rg=50

# Inicializar estructuras necesarias para la seleccion de cluster optimo
varianza_intra_cluster = []
silhouettes = []

# iterar entre todos los posibles clusters seleccionados para encontrar el optimo
for k in range (1, rg): # Evaluamos entre 1 a rg clusters posibles
    kmeans = KMeans(n_clusters = k, random_state = 42, n_init=50).fit(df_X)
    varianza_intra_cluster.append(kmeans.inertia_)
    try:
        silhouette = silhouette_score(df_X, kmeans.labels_)
    except:
        silhouette = 0 # El índice de Silhouette sólo se puede calcular cuando se tiene K>1
    silhouettes.append(silhouette)

In [ ]:
# Graficar la varianza intra cluster por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), varianza_intra_cluster, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Varianza intra clúster')
plt.show()

In [ ]:
# Graficar el indice de sihoutette por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), silhouettes, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Índice de Silhouette')
plt.grid()
plt.show()

### K-Medoides

In [ ]:
# Numero de cluster a validar
rg=50

# Inicializar estructuras necesarias para la seleccion de cluster optimo
varianza_intra_cluster = []
silhouettes = []

# iterar entre todos los posibles clusters seleccionados para encontrar el optimo
for k in range (1, rg): # Evaluamos entre 1 a rg clusters posibles
    kmedoids = KMedoids(n_clusters = k, random_state = 42).fit(df_X)
    varianza_intra_cluster.append(kmedoids.inertia_)
    try:
        silhouette = silhouette_score(df_X, kmedoids.labels_)
    except:
        silhouette = 0 # El índice de Silhouette sólo se puede calcular cuando se tiene K>1
    silhouettes.append(silhouette)

In [ ]:
# Graficar la varianza intra cluster por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), varianza_intra_cluster, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Varianza intra clúster')
plt.show()

In [ ]:
# Graficar el indice de sihoutette por cada uno de las posibles combinaciones de cluster a validar
plt.plot(range(1, rg), silhouettes, marker='o')
plt.xlabel('Número de clústeres (K)')
plt.ylabel('Índice de Silhouette')
plt.grid()
plt.show()

## Mejor modelo

In [ ]:
kmeans_2 = KMeans(n_clusters = 6, random_state = 42, n_init=50).fit(df_X)

In [ ]:
crear_grafico_dispersion(kmeans_2, df_X, df_y)

In [ ]:
print("Metrica de silhouette_score: ",silhouette_score(df_X, kmeans_2.labels_))
print("Metrica de Calinski-Harabasz Index: ", calinski_harabasz_score(df_X, kmeans_2.labels_))
print("Metrica de Inercia de k-means: ",kmeans_2.inertia_)

# Analisis Clusters - Mejor Modelo

## Analisis - clusters a data set original

In [ ]:
# Obtener las etiquetas de cluster generadas por KMeans
X_clusters = df7
X_clusters.drop(columns=['PB_DANE', 'CodigoD'], inplace=True)
X_clusters.insert(0, 'clusters', kmeans_2.labels_)

# ver df con labels de clusters
X_clusters.head()

In [ ]:
# Revisar los municipios con mayor indice en que grupo quedaron
X_clusters.sort_values(by='INDICADOR', ascending=False).head(20)

In [ ]:
# describe de los grupos
X_clusters.groupby('clusters').describe()

## Conclusion - Clusters

Se identifican 6 clusters de los cuales se resaltan el cluster 3 con un solo municipio el cual es el distrito capital Bogota el cual no sera tomando en cuenta ya que su desarrollo y la inclusion digital que este presenta tiene varios factores especiales con respecto a los demas.

El cluster 5 conformado por 10 municipios los cuales presentan un indicador promedio de 0.33 y un particular porcentaje de participacion en educacion muy bajo siendo de 0.04 en promedio.

y por ultimo el cluster 1 con 22 municipios el cual presenta un indicador promedio de 0.28 y un porcentaje de participacion de educacion mucho mejor el cual es de 0.59 en promedio.

con estos dos clusters podemos evidenciar las 2 situaciones contrarias en el pais donde el indicador ya se encuentra posicionado y con los correctos planes o iniciativas en cada uno podemos mejorar la integracion con las comunidades teniendo en cuenta las necesidades y el contexto donde se encuentran los 2 clusters.